<a href="https://colab.research.google.com/github/RemyaVKarthikeyan/AA-Stagecoach-Project/blob/main/File_Share_11_Fetching_the_time_table_of_a_given_stop_point_of_given_lineID%2C_direction_(based_on_day_of_the_week).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fetching Time table based on the day of the week (-based on day of the week-changing the time in the timetabe to HR: MIN format)- dividing to slots based on HR - Finding the number of scheduled buses in the given hr considering the present time and displaying the time table slots corresponding to the present HR, Hr-1 and HR+1

In [3]:
import requests
from datetime import datetime
import pytz

# Function to fetch data from the API
def fetch_data(url):
    response = requests.get(url)
    return response.json()

# Function to extract schedule names
def extract_schedule_names(data, schedule_names_dict={}):
    if isinstance(data, dict):
        if data.get('$type') == "Tfl.Api.Presentation.Entities.Schedule, Tfl.Api.Presentation.Entities" and 'knownJourneys' in data:
            if 'name' in data:
                schedule_names_dict[data['name']] = data['knownJourneys']
        for key, value in data.items():
            extract_schedule_names(value, schedule_names_dict)
    elif isinstance(data, list):
        for item in data:
            extract_schedule_names(item, schedule_names_dict)
    return schedule_names_dict

# Function to categorize journeys into hourly slots
def categorize_into_slots(timetable):
    slots = [[] for _ in range(24)]
    for journey in timetable:
        hour = int(journey['hour'])  # Convert hour to integer
        if 0 <= hour < 24:  # Ensure hour is within the valid range
            slots[hour].append(journey)
    return slots

# Function to calculate the number of buses between the first time available in the slot and the current time
def calculate_buses(slot, current_hour, current_minute):
    bus_count = 0
    for journey in slot:
        journey_hour = int(journey['hour'])
        journey_minute = int(journey['minute'])
        if journey_hour == current_hour and journey_minute <= current_minute:
            bus_count += 1
    return bus_count

# Main function to get user input and print the schedule names and timetable
def main():
    line_id = input("Enter the line ID: ")
    stop_point_id = input("Enter the stop point ID: ")
    direction = input("Enter the direction (outbound/inbound): ")

    # Construct the URL
    url = f'https://api.tfl.gov.uk/Line/{line_id}/Timetable/{stop_point_id}?direction={direction}'

    # Fetching data
    data = fetch_data(url)

    # Extracting schedule names
    schedule_names_dict = extract_schedule_names(data)

    # Printing the extracted schedule names as a numbered list
    print("Schedule Names:")
    schedule_names = list(schedule_names_dict.keys())
    for i, name in enumerate(schedule_names, start=1):
        print(f"{i}. {name}")

    # Asking the user to select a schedule
    selected_number = int(input("Enter the number corresponding to the schedule name (which matches with the day of the week): "))
    selected_schedule_name = schedule_names[selected_number - 1]

    # Categorizing the timetable into hourly slots
    timetable = schedule_names_dict[selected_schedule_name]
    slots = categorize_into_slots(timetable)

    # Get the current time in BST
    bst = pytz.timezone('Europe/London')
    now = datetime.now(bst)
    current_hour = now.hour
    current_minute = now.minute

    # Calculate the number of buses scheduled between the first time available in the current slot and the current time
    bus_count = calculate_buses(slots[current_hour], current_hour, current_minute)
    print(f"\nNumber of buses scheduled between the first time available in the current slot and now: {bus_count}")

    # Print the timetable slots corresponding to the current hour, the previous hour, and the next hour
    hours_to_print = [current_hour - 1, current_hour, current_hour + 1]
    print("\nTimetable:")
    print("Time   Line ID  Stop Point ID  Direction")
    for hour in hours_to_print:
        if 0 <= hour < 24:
            print(f"\nSlot {hour}:")
            for journey in slots[hour]:
                journey_hour = str(journey['hour']).zfill(2)
                journey_minute = str(journey['minute']).zfill(2)
                time = f"{journey_hour}:{journey_minute}"
                print(f"{time}   {line_id}    {stop_point_id}  {direction.capitalize()}")

# Run the main function
if __name__ == "__main__":
    main()


Enter the line ID: 179
Enter the stop point ID: 490005183E
Enter the direction (outbound/inbound): outbound
Schedule Names:
1. Mon-Fri Non-Schooldays
2. Mon-Fri Schooldays
3. Saturday
4. Sunday
Enter the number corresponding to the schedule name (which matches with the day of the week): 2

Number of buses scheduled between the first time available in the current slot and now: 3

Timetable:
Time   Line ID  Stop Point ID  Direction

Slot 16:
16:10   179    490005183E  Outbound
16:23   179    490005183E  Outbound
16:35   179    490005183E  Outbound
16:48   179    490005183E  Outbound

Slot 17:
17:00   179    490005183E  Outbound
17:13   179    490005183E  Outbound
17:25   179    490005183E  Outbound
17:37   179    490005183E  Outbound
17:49   179    490005183E  Outbound

Slot 18:
18:01   179    490005183E  Outbound
18:13   179    490005183E  Outbound
18:25   179    490005183E  Outbound
18:37   179    490005183E  Outbound
18:49   179    490005183E  Outbound
